In [2]:
import logging
import os
import numpy as np
from medpy.io import load
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow.keras import layers
import keras
import sklearn
from scipy.ndimage import rotate
import scipy
import time
from sklearn import datasets, preprocessing, model_selection, metrics
import matplotlib.pyplot as plt
%matplotlib inline

logging.basicConfig(level = 'logging.INFO')


In [3]:
'''Predetermined Functions'''
def Generic_image_generator(URL):
    image_data, image_header = load(URL)
    for iter in range(0,len(image_data[:][:])):
        yield image_data[iter]

def create_ID_dict(basepath): #currently only set to return CT and segmentation mask images, not CTA's
    '''basepath is location of file that is holding dataset, datasets should be deeper in the file tree'''
    logging.info("creating dataset location directory/dictionary from {}".format(basepath))
    dict = {}
    mask_list = []
    image_list = []
    for root, dirs, list_of_files in os.walk(basepath):
        for files in list_of_files:
            if "r.mhd" in files:
                mask_list.append(os.path.join(root,files))
            else: 
                pass
            
            if "cti.mhd" in files:
                image_list.append(os.path.join(root,files))
            else:
                pass
            
            
    dict.update({"image" : image_list })
    dict.update({"mask" : mask_list })
    logging.info("dictionary created")
    
    return dict
    
    
def image_generator(dict_obj, mask):
    
    generate_mask = False
    logging.info("creating image generator for {}. Images are Segmentation Masks? = {}.".format(dict_obj,mask))
    
    if mask == True:
        generate_mask = True
    
    if generate_mask == True:
        for path in dict_obj['mask']:
            logging.info("loading {} {} into generator".format("mask",os.path.basename(path)))
            image, header = load(path)
            image = rotate(image, 90, axes=(0,2), reshape=True) #rotates to axial form sup to inf
            logging.info("Generator created")
            yield image
            
    elif generate_mask == False:
        for path in dict_obj["image"]:
            logging.info("loading {} {} into generator".format("image",os.path.basename(path)))
            image, header  = load(path)
            image = rotate(image, 90, axes=(0,2), reshape=True) #rotates to axial form sup to inf
            logging.info("Generator created")
            yield image
    else:
        raise TypeError("Problem with inputs.")
        logging.error("Unable to load image into generator")
            
    #no normalization has been added yet. Will need to think about how I want to approach this.
    #from skimage.color import rgb2gray to make things grayscale.
            
    

In [ ]:
"""Need to do"""
'''
- Do I need to make a second generator for batch size? (https://www.kaggle.com/abhmul/python-image-generator-tutorial) 

'''

In [4]:
'''Predetermined Variables'''
PATH_tr = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Training_Set\\Train\\'
PATH_val = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Training_Set\\Train\\'
PATH_te = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Testing_Set\\'
Model_path = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\Coronary_Calcium\\Models\\'
model_vers = 'First_Attempt'

In [6]:
''' Loading Data Sets '''
logging.info("loading datasets")

start_time = time.time()
tr_dict = create_ID_dict(PATH_tr)
val_dict = create_ID_dict(PATH_val)
te_dict = create_ID_dict(PATH_te)
end_time = time.time()

print("Dictionary Creation Took {} seconds".format(end_time-start_time))

start_time = time.time()
tr_img = image_generator(tr_dict,mask = False)
tr_mask = image_generator(tr_dict, mask = True)
val_img = image_generator(val_dict, mask = False)
val_mask = image_generator(val_dict, mask = True)
te_img = image_generator(te_dict, mask = False)
end_time = time.time()


print("Generator Creation Took {} sec".format(end_time-start_time))

print("accessing shapes of tr_img images")
for image in tr_img:
    print(image.shape)
print("accessing shapes of tr_mask images")
for mask in tr_mask:
    print(mask.shape)
print("others")
for image in val_img:
    print(image.shape)
for mask in val_mask:
    print(mask.shape)
for image in te_img:
    print(image.shape)
    
'''
num = 0    
for image in tr_img:
    while num < 25:
        for slice in image:
            start_time = time.time()
            plt.imshow(slice)
            plt.show()
            end_time = time.time() 
            print("Displaying Image Slice took {} sec".format(end_time-start_time))
            
            num +=1
'''


Dictionary Creation Took 0.040990591049194336 seconds
Generator Creation Took 0.0 sec
accessing shapes of tr_img images
(64, 512, 512)
(56, 512, 512)
(56, 512, 512)
(64, 512, 512)
(48, 512, 512)
(56, 512, 512)
(56, 512, 512)
(56, 512, 512)
(47, 512, 512)
(47, 512, 512)
(43, 512, 512)
(47, 512, 512)
(47, 512, 512)
(47, 512, 512)
(47, 512, 512)
(47, 512, 512)
(39, 512, 512)
(41, 512, 512)
(41, 512, 512)
(41, 512, 512)
(41, 512, 512)
(43, 512, 512)
(54, 512, 512)
(46, 512, 512)
(48, 512, 512)
(54, 512, 512)
(42, 512, 512)
(42, 512, 512)
(48, 512, 512)
(48, 512, 512)
(36, 512, 512)
(48, 512, 512)
accessing shapes of tr_mask images
(64, 512, 512)
(56, 512, 512)
(56, 512, 512)
(64, 512, 512)
(48, 512, 512)
(56, 512, 512)
(56, 512, 512)
(56, 512, 512)
(47, 512, 512)
(47, 512, 512)
(43, 512, 512)
(47, 512, 512)
(47, 512, 512)
(47, 512, 512)
(47, 512, 512)
(47, 512, 512)
(39, 512, 512)
(41, 512, 512)
(41, 512, 512)
(41, 512, 512)
(41, 512, 512)
(43, 512, 512)
(54, 512, 512)
(46, 512, 512)
(48, 

KeyboardInterrupt: 

In [16]:
''' Creating NN'''


if os.path.isdir(Model_path+model_vers)==True:
    model=tfk.models.load_model("Model_path+model_vers")
    logging.info("Loading Network")
else:
    pass
   
''' For a 2D network'''
"""model= tfk.Sequential()
    model.add(Conv2D(64, kernel_size=3, stride=(1,1), activation=’relu’, input_shape=(28,28,1)))
    model.add(Conv2D(32, kernel_size=3,stride=(1,1), activation=’relu’))
    model.add(Flatten())
    model.add(Dense(10, activation=’softmax’))"""
''' For a 3D network'''
    #tf.keras.layers.Conv3D(filters, kernel_size, strides=(1, 1, 1), padding='valid', data_format=None,
    #dilation_rate=(1, 1, 1), activation=None, use_bias=True,
    #kernel_initializer='glorot_uniform', bias_initializer='zeros',
    #kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
    #kernel_constraint=None, bias_constraint=None, **kwargs)
logging.info("Creating Network")
model= tfk.Sequential()
input_shape = (64,512,512,1,1)
model.add(layers.Conv3D(filters=64, kernel_size=3,strides=(1, 1, 1), activation='relu', input_shape=input_shape))
model.add(layers.Conv3D(filters=32, kernel_size=3,strides=(1, 1, 1), activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

'''Hyper Parameters'''
logging.info("Adding Hyperparameters")

optimizer=tfk.optimizers.Adadelta()
loss="categorical_crossentropy"
metrics = ['accuracy']
model.compile(optimizer = optimizer, loss= loss, metrics= metrics )


ValueError: Input 0 of layer conv3d is incompatible with the layer: expected ndim=5, found ndim=6. Full shape received: [None, 64, 512, 512, 1, 1]

In [ ]:
"""Training NN"""
logging.info("Training Neural Network")
model.compile(optimizer = optimizer, loss= loss, metrics= metrics )
model.save("First_Attempt_Model/Model")
